In [ ]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()

In [ ]:
import numpy as np
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

In [ ]:
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [ ]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")
ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()


In [ ]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [ ]:
test_data = pd.read_csv("dev/test_2_features.csv")
test_metadata = pd.read_csv("dev/test_2_metadata.csv")
ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

In [ ]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
ana.predict(test2_data, test2_metadata)
outliers_test2 = ana.test_prediction_outliers()
print(outliers_test2)
fig_test2_scores = ana.plot_scores()
fig_test2_scores.write_image("dev/figures/fig_test2_scores.png", width=1100, height= 350, scale = 3)
fig_test2_scores.show()

In [ ]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()

In [ ]:
#ana.add_prediction(add_outliers = False)
#ana.plot_scores()

In [ ]:
# test_2_data = pd.read_csv("test_2_features.csv")
# test_2_metadata = pd.read_csv("test_2_metadata.csv")
# ana.predict(test_2_data, test_2_metadata)
# outliers_test_2 = ana.test_prediction_outliers()
# print(outliers_test_2)
# fig_test_scores=ana.plot_scores()
# fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1000, height= 350, scale = 3)
# fig_test_scores.show()

In [ ]:
#ana.add_prediction(add_outliers = False)
#ana.plot_scores()